In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('fare_rules.txt')

In [4]:
data

,fare_id,route_id,origin_id,destination_id
0,DIMTS_0_3127_2620,0,3127,2620
1,DIMTS_0_3127_2020,0,3127,2020
2,DIMTS_0_3127_2021,0,3127,2021
3,DIMTS_0_3127_4120,0,3127,4120
4,DIMTS_0_3127_2023,0,3127,2023
...,...,...,...,...
2751907,DTC_6001_1604_1306,6001,1604,1306
2751908,DTC_6001_1604_68,6001,1604,68
2751909,DTC_6001_1605_1306,6001,1605,1306
2751910,DTC_6001_1605_68,6001,1605,68


In [6]:
from collections import defaultdict
d = defaultdict(list)

In [7]:
for i in range(len(data)):
    routeid = data['route_id'][i]
    origin = data['origin_id'][i]
    destination = data['destination_id'][i]

    if routeid not in d:
        d[routeid].append(origin)
        d[routeid].append(destination)
    else:
        if origin==d[routeid][0]:
            d[routeid].append(destination)


In [15]:
stops = pd.read_csv('stops.txt')
distance = pd.read_csv('DistanceMatrix.txt',header=None)

In [14]:
fake_real = {}
real_fake = {}
for i in range(len(stops)):
    fake_real[i] = stops['stop_id'][i]
    real_fake[stops['stop_id'][i]] = i

In [36]:
stops_matrix = np.zeros((len(stops),len(stops)))

In [37]:
for i in d:
    lst = d[i]
    start = lst[0]
    route = [start]
    s = set(lst[1:])


    visited = {start}
    for i in visited:
        if i in s:
            s.remove(i)

            
    while s:
        mn_dis = float('inf')
        mn_stop = None
        for stop in s:
            if distance[real_fake[start]][real_fake[stop]] < mn_dis:
                mn_dis = distance[real_fake[start]][real_fake[stop]]
                mn_stop = stop
        route.append(mn_stop)
        s.remove(mn_stop)
        start = mn_stop
    for i in range(len(route)-1):
        stops_matrix[real_fake[route[i]]][real_fake[route[i+1]]] = 1



In [38]:
for i in range(len(stops_matrix)):
    for j in range(len(stops_matrix)):
        if stops_matrix[i][j]==1 or i==j:
            continue
        if distance[i][j]<0.1:
            stops_matrix[i][j] = 2

In [39]:
stops_matrix_df = pd.DataFrame(stops_matrix,columns=stops["stop_id"],index=stops["stop_id"])
stops_matrix_df = stops_matrix_df.astype(int)
stops_matrix_df.to_csv("stops_matrix.csv")

In [40]:
df = pd.read_csv('stops_matrix.csv')

In [41]:
df

,stop_id,0,1,2,3,4,5,6,7,8,...,17109,17110,17111,17112,17113,17114,17115,17116,17117,17118
0,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,17114,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6625,17115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6626,17116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6627,17117,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### making ```stops_matrix.csv``` using the data in ```stop_times.txt```

In [42]:
stop_times = pd.read_csv('stop_times.txt')

In [47]:
# stop_times

In [48]:
route_stands = defaultdict(set)
route_start = defaultdict(int)

In [49]:
for i in range(len(stop_times)):
    trip_id = stop_times['trip_id'][i]
    route_id = trip_id.split('_')[0]
    if route_id not in route_start:
        route_start[route_id] = stop_times['stop_id'][i]
    route_stands[route_id].add(stop_times['stop_id'][i])


In [50]:
len(route_stands),len(route_start)

(3155, 3155)

In [51]:
stops_matrix = np.zeros((len(stops),len(stops)))

In [52]:
for i in route_stands:

    start = route_start[i]
    route = [start]
    s = route_stands[i]

    visited = {start}
    for i in visited:
        if i in s:
            s.remove(i)

    while s:
        mn_dis = float('inf')
        mn_stop = None
        for stop in s:
            if distance[real_fake[start]][real_fake[stop]] < mn_dis:
                mn_dis = distance[real_fake[start]][real_fake[stop]]
                mn_stop = stop
        route.append(mn_stop)
        s.remove(mn_stop)
        start = mn_stop
        
    for i in range(len(route)-1):
        stops_matrix[real_fake[route[i]]][real_fake[route[i+1]]] = 1



In [53]:
for i in range(len(stops_matrix)):
    for j in range(len(stops_matrix)):
        if stops_matrix[i][j]==1 or i==j:
            continue
        if distance[i][j]<0.1:
            stops_matrix[i][j] = 2

In [54]:
stops_matrix_df = pd.DataFrame(stops_matrix,columns=stops["stop_id"],index=stops["stop_id"])
stops_matrix_df = stops_matrix_df.astype(int)
stops_matrix_df.to_csv("stops_matrix.csv")

df = pd.read_csv('stops_matrix.csv')
df

,stop_id,0,1,2,3,4,5,6,7,8,...,17109,17110,17111,17112,17113,17114,17115,17116,17117,17118
0,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6624,17114,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6625,17115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6626,17116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6627,17117,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
